In [1]:
import pandas as pd

In [2]:
df_classifier_test = pd.read_csv('data/submissions/classifier_test.csv').fillna('')

In [3]:
df_classifier_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


## pseudo labing from classifier predictions 

In [4]:
df_classifier_test_confidence = pd.read_csv('data/classifier_test_confidence.csv').fillna('')

In [5]:
df_classifier_test_confidence.head()

,ImageId_ClassId,Confidence
0,004f40c73.jpg,0.016510
1,006f39c41.jpg,0.045337
2,00b7fb703.jpg,0.176907
3,00bbcd9af.jpg,0.001386
4,0108ce457.jpg,0.277736


In [6]:
print(len(df_classifier_test_confidence[df_classifier_test_confidence.Confidence<0.05]))
print(len(df_classifier_test_confidence[df_classifier_test_confidence.Confidence>0.8]))

1404
196


I'll add the segmentation model predictions to the train dataset that have 
- < 5% confidence on negatives and 
- \> 80% confidence on positives

in the classification model.

In [7]:
df_classifier_confident = df_classifier_test_confidence[(df_classifier_test_confidence.Confidence<0.05) | 
                                                        (df_classifier_test_confidence.Confidence>0.8)]

In [8]:
df_classifier_confident = df_classifier_confident.reset_index(drop=True)

In [53]:
df_classifier_confident.to_csv('data/confident_classification.csv', index=False)

In [54]:
#df_segmentation_test_confidence = pd.read_csv('data/segmentation_test_confidence.csv').fillna('')

In [55]:
#df_segmentation_test_confidence.head()

In [9]:
df_segmentation_test = pd.read_csv('data/submissions/segmentation_test.csv').fillna('')

In [10]:
df_segmentation_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [11]:
df_train = pd.read_csv('data/train.csv').fillna('')

In [12]:
df_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [14]:
pseudolabel = []

for ind, row in df_classifier_confident.iterrows():
    if row['Confidence'] < 0.05 or row['Confidence'] > 0.8:
        for i in range(4):
            class_id = row.ImageId_ClassId + '_' + str(i+1)

            rle = df_segmentation_test.loc[df_segmentation_test.ImageId_ClassId == class_id].EncodedPixels.values
            if len(rle) > 0:
                rle = rle[0]
            else:
                rle = None
            pseudolabel.append({
                'ImageId_ClassId': class_id,
                'EncodedPixels': rle,
            })

In [15]:
df_pseudolabel_test = pd.DataFrame(pseudolabel)

In [16]:
df_pseudolabel_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [19]:
len(df_pseudolabel_test), len(df_train)

(6400, 50272)

In [20]:
df_pseudolabel_test[df_pseudolabel_test.EncodedPixels != '']

,ImageId_ClassId,EncodedPixels
10,00bbcd9af.jpg_3,72020 7 72271 20 72522 27 72775 32 73029 36 73...
15,0109b68ec.jpg_4,154353 8 154603 16 154854 22 155108 25 155362 ...
18,010ec96b4.jpg_3,38140 1 38375 24 38617 2 38621 35 38718 113 38...
30,01b47d973.jpg_3,245196 4 245450 7 245705 8 245960 10 246215 12...
34,01d49cd47.jpg_3,297610 1 297615 2 297855 41 297961 2 297966 7 ...
...,...,...
6362,fe3116441.jpg_3,308699 6 308956 4 344437 79 344552 24 344683 1...
6370,ff18d8aa8.jpg_3,31807 4 32061 8 32313 13 32568 15 32823 17 330...
6374,ff4d29e0d.jpg_3,275714 17 275969 69 276225 74 276481 76 276737...
6390,ffaf13d96.jpg_3,76779 1 76783 15 76999 56 77241 70 77483 84 77...


In [21]:
df_pseudo_train = df_train.append(df_pseudolabel_test, ignore_index=True)

In [22]:
df_pseudo_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [23]:
len(df_pseudo_train), len(df_train), len(df_pseudolabel_test)

(56672, 50272, 6400)

In [101]:
df_pseudo_train.to_csv('data/train_pseudolabel.csv', index=False)